# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [ ]:
# unzip ./data/useful_data.zip

In [1]:
# !pip install tqdm rich

In [2]:
import re
from collections import defaultdict

import rich  # for beatifull output formatting
from tqdm import tqdm

In [67]:
# reading Ngram data
def read_n_bigrams(path: str) -> tuple[dict[tuple[str] : int], set]:
    data = defaultdict(int)
    vocab = set()
    with open(path, "r", encoding="latin1") as file:
        for line in tqdm(file):
            line_ = line.lower().strip().split("\t")

            # padding simulation
            ln = len(line_) - 2
            tpl = tuple(["<s>"] * ln + line_[1:] + ["<s>"] * ln)
            for idx in range(len(tpl) - ln):
                data[tpl[idx : len(line_) - 1 + idx]] = int(line_[0])

            for elem in line_[1:]:
                vocab.add(elem)
    return data, vocab


bigrams, vocab_bigrams = read_n_bigrams("./data/bigrams.txt")
fivegrams, vocab_fivegrams = read_n_bigrams("./data/fivegrams.txt")

vocab = vocab_bigrams & vocab_fivegrams
len(bigrams), len(fivegrams)

1020385it [00:02, 482455.45it/s]
1044268it [00:05, 196437.16it/s]


(1130077, 3205806)

In [68]:
import math


class NGramModel:
    def __init__(
        self, ngram_count: dict[tuple[str], int], vocab: set, ngram_size: int = 2
    ):
        self.ngram_count = ngram_count
        self.vocab = vocab
        self.ngram_size = ngram_size

        self.prev_grams = defaultdict(int)
        for ngram, count in ngram_count.items():
            self.prev_grams[ngram[:-1]] += count

        self.probs = {}
        for ngram, count in ngram_count.items():
            self.probs[ngram] = (count + 1) / self.prev_grams[ngram[:-1]]

    def evaluate_ngram(self, sequence, verbose: bool = False):
        probability = 0
        for i in range(len(sequence) - self.ngram_size + 1):
            ngram = tuple(sequence[i : i + self.ngram_size])
            probability += math.log(self.probs.get(ngram, 1e-10))

            if verbose:
                rich.print(ngram, math.log(self.probs.get(ngram, 1e-10)), probability)

        return probability

In [99]:
class SpellCorrector:
    def __init__(
        self,
        vocab: set,
        ngrams: list[dict[tuple, int]],
        max_dist: int = 20,
        le: float = 0.5,
        lm_1: float = 0.5,
        lm_2: float = 0.5,
    ):
        self.vocab = vocab
        self.lm = None
        self.max_dist = max_dist

        self.ngrams = ngrams
        self.ngram_models = [
            NGramModel(ngram_dict, self.vocab, ngram_size=ngram_size)
            for ngram_dict, ngram_size in zip(self.ngrams, [2, 5])
        ]

        self.le = le
        self.lm_1 = lm_1
        self.lm_2 = lm_2

    def dameru_levenshtein_distance(self, a: str, b: str) -> tuple[int, float]:
        # source: https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance
        m, n = len(a), len(b)
        max_dist = m + n

        alphabet = set(a) | set(b)
        da = [0 for _ in range(len(alphabet) + 1)]
        char2idx = {char: 0 for char in alphabet}

        d = [[0 for i in range(n + 2)] for j in range(m + 2)]
        d[0][0] = max_dist

        for i in range(0, m + 1):
            d[i + 1][0] = max_dist
            d[i + 1][1] = i

        for j in range(0, n + 1):
            d[0][j + 1] = max_dist
            d[1][j + 1] = j

        a, b = " " + a, " " + b
        for i in range(1, m + 1):
            db = 0
            min_cost = float("inf")
            for j in range(1, n + 1):
                k = da[char2idx[b[j]]]
                l = db
                if a[i] == b[j]:
                    cost = 0
                    db = j
                else:
                    cost = 2
                substitution = d[i][j] + cost
                insertion = d[i + 1][j] + 1
                deletion = d[i][j + 1] + 1
                d[i + 1][j + 1] = min(substitution, insertion, deletion)

                # trying to not doing extra calculations
                if i > 1 and j > 1 and a[i - 1] == b[j - 2] and a[i - 2] == b[j - 1]:
                    d[i + 1][j + 1] = min(d[i + 1][j + 1], d[k][l] + cost)

                min_cost = min(d[i + 1][j + 1], min_cost)

            if min_cost >= self.max_dist:
                return self.max_dist + 1

            da[char2idx[a[i]]] = i

        return d[m + 1][n + 1]

    def __generate_candidates(self, pretendant: str, top_k: int = 20):
        candidates = []
        n = len(pretendant)
        pret_set = set(pretendant)
        for word in self.vocab:
            if abs(len(word) - n) > self.max_dist:
                continue
            if len(pret_set.intersection(set(word))) < (len(pret_set) / 2):
                continue

            dist = self.dameru_levenshtein_distance(pretendant, word)
            if dist <= self.max_dist:
                candidates.append((word, dist))

        return sorted(candidates, key=lambda x: x[1])[:top_k]

    def __evaluate_candidates(
        self,
        sentence: list[str],
        candidates: tuple[list[str], int],
        candidate_idx: int,
        verbose: bool,
    ) -> str:
        best = None
        best_score = -float("inf")

        for candidate, dist in candidates:
            scores = {}
            for model in self.ngram_models:
                start_idx, end_idx = (
                    max(candidate_idx - model.ngram_size + 1, 0),
                    min(candidate_idx + model.ngram_size, len(sentence)),
                )

                sent_copy = sentence.copy()
                sent_copy[candidate_idx] = candidate

                left_slice = sent_copy[start_idx : candidate_idx + 1]
                right_slice = sent_copy[candidate_idx:end_idx]
                left = len(sent_copy[start_idx : candidate_idx + 1])

                if left != model.ngram_size:
                    left_slice = ["<s>"] * (model.ngram_size - left) + sent_copy[
                        start_idx : candidate_idx + 1
                    ]

                sent = left_slice + right_slice[1:]

                scores[model.ngram_size] = model.evaluate_ngram(
                    sequence=sent, verbose=verbose
                )

            combined_score = (
                -self.le * dist + self.lm_1 * scores[2] + self.lm_2 * scores[5]
            )
            if verbose:
                rich.print(
                    f"{candidate}\t{combined_score=}\t", dist, scores[2], scores[5]
                )

            if combined_score > best_score:
                best_score = combined_score
                best = candidate

        return best

    def spell_checker(
        self, text: str, top_k: int = 10_000, verbose: list[bool] = [False, False]
    ) -> str:
        words = re.findall(r"\w+", text.lower())
        corrected_sentence = []

        for idx, word in enumerate(words):
            corrected_word = word
            if word not in self.vocab:
                candidates = self.__generate_candidates(pretendant=word, top_k=top_k)

                best_one = self.__evaluate_candidates(
                    sentence=words,
                    candidates=candidates,
                    candidate_idx=idx,
                    verbose=verbose[0],
                )
                corrected_word = best_one
            elif verbose[1]:
                candidates = self.__generate_candidates(pretendant=word, top_k=top_k)
                ecand = self.__evaluate_candidates(
                    sentence=words,
                    candidates=candidates,
                    candidate_idx=idx,
                    verbose=verbose[1],
                )
                rich.print("non-changed:\n", ecand, "\n\n")
            words[idx] = corrected_word
            corrected_sentence.append(corrected_word)
        return " ".join(corrected_sentence)


top_k = 10000
verbose = [False, False]
spell_corrector = SpellCorrector(vocab=vocab, ngrams=[bigrams, fivegrams], max_dist=10)
(
    spell_corrector.spell_checker("fml sport", verbose=verbose, top_k=top_k),
    spell_corrector.spell_checker("mle sport", verbose=verbose, top_k=top_k),
    # 'these are / this is' are seen by model
    spell_corrector.spell_checker("these aris", verbose=verbose, top_k=top_k),
    spell_corrector.spell_checker("this aris", verbose=verbose, top_k=top_k),
    spell_corrector.spell_checker(
        "dking sports", verbose=verbose, top_k=top_k
    ),  # see explanations: corpora does not have such pairs
    spell_corrector.spell_checker("dking animals", verbose=verbose, top_k=top_k),
    spell_corrector.spell_checker(
        "encnhanceded the quaility of slewp", verbose=verbose, top_k=top_k
    ),
)

('female sport',
 'male sport',
 'these are',
 'this is',
 'organized sports',
 'infected animals',
 'enhance the quality of sleep')

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### My approach

I used following idea: 

- per each single word we can find it's _corrections_. By _corrections_ I mean finding the words into wich the word could be changed (based on data of unigrams I collect). By doing so, I will retrieve some top-k words that could be 'derived' by correcting the word: most probably the _true_ word will be in the top-k ones, even if the initial word is correct (but I assume that the wrong words a those that not included in _vocab_ that model was buid on).

- Using NGram Language Model, pick one word from top-k ones, chosen in prev. step (with some number of surrounding words) and compute the probabilities that model gives for the chosen word to be in the sentence. Pick the best word based on highest probability.

Potentially these actions could be vectorized, so some speed-up could be gain.

### What I need to implement

- Collect unigrams for single word spell corrector
- Collect NGrams for Ngram LM
- Unigram corrector: 
    - Distance to see 'similarity' words (it could be named differently)
- NGram LM:
    - Train the model on the data

- Evaluator to evaluate the solution:
    - 'Correctness' of spellcheking
    - 'Time' used for the algorithm

### Difficulties

The 'top-k' approach with the general distance computation for pairs of words (even with possible improvements) work quite slowly. I see the only way to fix it: use the other way of singe-word-correction

My implementation does not handle _wrong word by meaning_, since I check for words to correct initially based on grammar / syntax, then I pick the most suitable word using NLM predictions  

### What and Why I have used

Word change distance: 

As I know, three main algorithms used: Levenshtein, Damerau–Levenshtein and Optimal String Allignment (OSA).
Other algortithms, like Hamming distance, etc. seem to be used in some specifiec scenarios, not the general spelling correction.

My idea was to use an algorithm, that is capable of fixing 3+ mistaken letter, therefore I used Damerau–Levenshtein (but it's quite computation expensive)

The Damerau–Levenshtein:

- Insert / Delete / Swap operations, as in Levenshtein distance or in OSA
- Transpositions (or order swap of characters)
    - Adjacent transposition: adjacent characters in word swapped positions - typo fix (as in OSA)
    - Overlapping transposition: handle cases with keyboard misstyping - only the Damerau–Levenshtein supports it
- The algorithm is about _O(mn)_, but it could be speeded up using tree-based structures to about _O(m log n)_ comlexity (not implemented)


The Damerau–Levenshtein distance is optimised by early stopping by max edit distance:

- In some way, the Norvig approach is faster thanks to 2-edit based words, since the number of comparisons \ operations is in magnitude lower, but with the cases of higher changes / transposes Norvig does no work, while distance calculation works

My solution is quite slow compared to the Norvig on larger sentences (moreover, I use in some sense 1-2-3-4-5 gram models implicinty by creating paddings), but my optimisations slightly speeded-up the algorithm

The _top_k_ parameter increase does not signigicanly slows the model's performance, this increases the overall generalization of the model's corrections (sometimes it's more like an word changer)  

Algorithm is capable of correcting multiple words in sentence

##### Ngram Language Models

For Ngram Language Model I used rough estimating of probabilities, since building matrix of ngram, as we usually made is too memory-consuming for such large vocablurary size: the ordinaty solution to create matrix is generally impossible for n>3, as more than 18GB RAM needed (tested this to compare RAM usage of approaches). Even bigram table uses near 8GB RAM with these data, while dicrionary-based 2 and 5 gram models need less than 800MB of RAM.

My implementation uses 2 and 5 gram language models, with "Model Interpolation" approach in combining the results of estimations. 


#### Data

I do not clean data in sense, that stopwords (that usually removed) could be used to correct. Moreover, I assumed to  lowercase all words, since small number of original words is capitalized (but mostly vocab I use is lowercased, this might be some issue) and therefore I will get high number of 'unk tokens' for predictions

##### I used the bigrams, fivegrams dataset from moodle

- The 'dking' example does from description not work, since corpora does not have these ones
- In the brief test case near the SpellChecker was implemented, I showed some pair-cases that are working, since similar data was in corpora


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

### Results

For evaluation I simply use NLTK's dataset, wich is already structured text and it is easy to download.

Unfortunatelly, I could not 'beat' the Norvig's performance in the test set, but my model seems to be more robust, since the accuracy drop is not that significant, as Norvig's. 

I used fixed seed - compared to some random seed, this is hard seed, since usually Norvig's scores higher than 0.7, and my model lower by ~0.2 points every time 

'By eye approximation' - with good enought educational vocablurary, some context - based addition appearence of words. For cases with high errors probability (0.6 anf greater) my model has slighly higher accuracy if using with _top_k_ less than 100 (increased robustness, but overall with smaller _top_k_ the average performance drops)

--- Noise Probability: 0.01 ---

Norvig:
 - Word Acc: 0.58
 - Sent Acc: 0.58

My Model:
 - Word Acc: 0.26
 - Sent Acc: 0.26

--- Noise Probability: 0.1 ---

Norvig:
 - Word Acc: 0.38
 - Sent Acc: 0.38

My Model:
 - Word Acc: 0.2
 - Sent Acc: 0.2

--- Noise Probability: 0.4 ---

Norvig:
 - Word Acc: 0.08
 - Sent Acc: 0.08

My Model:
 - Word Acc: 0.06
 - Sent Acc: 0.06

 --- Noise Probability: 0.8 ---

Norvig:
 - Word Acc: 0.04
 - Sent Acc: 0.04
 
 My Model:
 - Word Acc: 0.02
 - Sent Acc: 0.02

In [6]:
# This part populates the file from ngram files
# Quite ineffective implementation + no lowering
def populate_with_text(ngram_sets: dict[tuple, int]) -> list[str]:
    text = defaultdict(int)
    for ngram_set in ngram_sets:
        for key, item in ngram_set.items():
            for word in key:
                if word != "<s>":
                    text[word] += item
    return text


big_text = populate_with_text(ngram_sets=[bigrams, fivegrams])
# End of populating

In [7]:
# Norvig's solution
# it's just copy-paste from web-page for fair comparison with my solution + added capital letters
import re
from collections import Counter


def words(text):
    return re.findall(r"\w+", text.lower())


WORDS = big_text  # Counter(words(open("big.txt").read()))


def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N


def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)


def candidates(word):
    "Generate possible spelling corrections for word."
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = "abcdefghijklmnopqrstuvwxyz"
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [8]:
# !pip install nltk

In [100]:
import random

import nltk
from nltk.corpus import brown

nltk.download("brown")
random.seed(124)


def add_noise(word: str, noise_prob: float = 0.1):
    if random.random() > noise_prob or len(word) < 2:
        return word

    ops = ["d", "i", "s", "t"]
    op = random.choice(ops)
    idx = random.randint(0, len(word) - 1)

    if op == "d":
        return word[:idx] + word[idx + 1 :]
    elif op == "i":
        return word[:idx] + random.choice(r"a-z") + word[idx:]
    elif op == "s":
        return word[:idx] + random.choice(r"a-z") + word[idx + 1 :]
    elif op == "t" and idx < len(word) - 1:
        return word[:idx] + word[idx + 1] + word[idx] + word[idx + 2 :]
    return word


def word_accuracy(original: list[str], corrected: list[str]):
    correct = sum(o == c for o, c in zip(original, corrected))
    total = len(original)
    return correct / total


def sentence_accuracy(original_sents: list[str], corrected_sents: list[str]):
    correct_sents = sum(o == c for o, c in zip(original_sents, corrected_sents))
    return correct_sents / len(original_sents)


def evaluate_models(model: SpellCorrector, noise_probs=[0.01, 0.1, 0.4, 0.8]):
    sentences = brown.sents(categories="news")[:50]

    for p in noise_probs:
        print(f"\n--- Noise Probability: {p} ---")
        dataset = [
            (" ".join(sent), " ".join([add_noise(word, p) for word in sent]))
            for sent in sentences
        ]
        original_sents = [
            " ".join(re.findall(r"\w+", orig.lower())) for orig, _ in dataset
        ]
        noisy_sents = [
            " ".join(re.findall(r"\w+", noisy.lower())) for _, noisy in dataset
        ]

        norvig_corrected = [
            " ".join(correction(w) for w in re.findall(r"\w+", sent))
            for sent in noisy_sents
        ]

        spell_cheker_corrected = [model.spell_checker(sent) for sent in noisy_sents]

        print("Norvig:")
        print(" - Word Acc:", word_accuracy(original_sents, norvig_corrected))
        print(" - Sent Acc:", sentence_accuracy(original_sents, norvig_corrected))

        print("My Model:")
        print(" - Word Acc:", word_accuracy(original_sents, spell_cheker_corrected))
        print(" - Sent Acc:", sentence_accuracy(original_sents, spell_cheker_corrected))

[nltk_data] Downloading package brown to /Users/ilia/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [101]:
evaluate_models(spell_corrector)


--- Noise Probability: 0.01 ---
Norvig:
 - Word Acc: 0.58
 - Sent Acc: 0.58
My Model:
 - Word Acc: 0.26
 - Sent Acc: 0.26

--- Noise Probability: 0.1 ---
Norvig:
 - Word Acc: 0.38
 - Sent Acc: 0.38
My Model:
 - Word Acc: 0.2
 - Sent Acc: 0.2

--- Noise Probability: 0.4 ---
Norvig:
 - Word Acc: 0.08
 - Sent Acc: 0.08
My Model:
 - Word Acc: 0.06
 - Sent Acc: 0.06

--- Noise Probability: 0.8 ---
Norvig:
 - Word Acc: 0.04
 - Sent Acc: 0.04
My Model:
 - Word Acc: 0.02
 - Sent Acc: 0.02


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)